In [1]:
from model import classic_model, losses, dot_prod_attention
from data import data_generation, batch_creator, gp_kernels
from keras.callbacks import ModelCheckpoint
from helpers import helpers, masks, metrics
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from inference import infer
import tensorflow as tf
import numpy as np
import matplotlib 
import time
import keras
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
import pandas as pd

In [3]:
attributes = pd.read_csv('/Users/omernivron/Downloads/attibutes.csv')

In [7]:
attributes_numeric = attributes[[#'gauge_id', 
               'p_mean', 'pet_mean', 'p_seasonality', 'frac_snow',
               'aridity', 'high_prec_freq', 'high_prec_dur',
                #'high_prec_timing',
               'low_prec_freq', 'low_prec_dur',
                #'low_prec_timing', 'geol_1st_class', 'glim_1st_class_frac', 'geol_2nd_class', 'glim_2nd_class_frac',
               'carbonate_rocks_frac', #'geol_porostiy',
                'geol_permeability',
                #'q_mean','runoff_ratio', 'slope_fdc', 'baseflow_index', 'stream_elas', 'q5',
               #'q95', 'high_q_freq', 'high_q_dur', 'low_q_freq', 'low_q_dur',
               #'zero_q_freq', 'hfd_mean', 'huc_02', 'gauge_name',
               'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity','soil_conductivity',
                'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac',
                #'water_frac', 'organic_frac', 'other_frac',
                #'gauge_lat','gauge_lon',
                'elev_mean', 'slope_mean',
                #'area_gages2',
                'area_geospa_fabric',
                'frac_forest', 'lai_max', 'lai_diff', 'gvf_max','gvf_diff',
                #'dom_land_cover_frac', 'dom_land_cover', 'root_depth_50','root_depth_99', 'hru08'
                ]]

In [9]:
attributes_numeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671 entries, 0 to 670
Data columns (total 27 columns):
p_mean                  671 non-null float64
pet_mean                671 non-null float64
p_seasonality           671 non-null float64
frac_snow               671 non-null float64
aridity                 671 non-null float64
high_prec_freq          671 non-null float64
high_prec_dur           671 non-null float64
low_prec_freq           671 non-null float64
low_prec_dur            671 non-null float64
carbonate_rocks_frac    671 non-null float64
geol_permeability       671 non-null float64
soil_depth_pelletier    671 non-null float64
soil_depth_statsgo      671 non-null float64
soil_porosity           671 non-null float64
soil_conductivity       671 non-null float64
max_water_content       671 non-null float64
sand_frac               671 non-null float64
silt_frac               671 non-null float64
clay_frac               671 non-null float64
elev_mean               671 non-null flo

In [21]:
selected_basins = pd.read_csv('/Users/omernivron/Downloads/basin_list.txt', header=None)

In [27]:
df = pd.read_csv('/Users/omernivron/Downloads/daymet_data_seed05.csv')

In [28]:
df.head()

,dayl(s),prcp(mm/day),srad(W/m2),swe(mm),tmax(C),tmin(C),vp(Pa),basin,YR,MNTH,...,seed,DATE,DOY,doy_sin,doy_cos,hru08,hru02,hru04,id,id_lag
0,40780.77,3.10,192.61,0.0,10.05,2.11,711.33,1013500,1980,10,...,5,1980-10-01,275,-0.017062,0.001896,hru_01013500,hru_01,hru_0101,hru_01013500_1980,hru_01013500_1979
1,40621.08,4.24,206.31,0.0,15.82,5.24,898.61,1013500,1980,10,...,5,1980-10-02,276,-0.007624,0.015382,hru_01013500,hru_01,hru_0101,hru_01013500_1980,hru_01013500_1979
2,40435.18,8.02,165.35,0.0,15.86,7.81,1064.86,1013500,1980,10,...,5,1980-10-03,277,0.008824,0.014726,hru_01013500,hru_01,hru_0101,hru_01013500_1980,hru_01013500_1979
3,40089.56,15.27,159.78,0.0,11.10,3.66,794.18,1013500,1980,10,...,5,1980-10-04,278,0.017159,0.000531,hru_01013500,hru_01,hru_0101,hru_01013500_1980,hru_01013500_1979
4,40089.56,8.48,133.83,0.0,7.58,2.02,705.50,1013500,1980,10,...,5,1980-10-05,279,0.009718,-0.014152,hru_01013500,hru_01,hru_0101,hru_01013500_1980,hru_01013500_1979


In [20]:
list_to_drop = ['MNTH', 'DY', 'hru02', 'hru04', 'RAIM', 'TAIR', 'PET', 'ET', 'SWE', 'swe(mm)', 'PRCP', 'seed', 'id_lag', 'HR', 'dayl(s)', 'YR']

In [22]:
df = df[df['basin'].isin(selected_basins)]
print("Length of dataframe for selected basin: ", len(df))
df.drop(['MOD_RUN'], axis=1, inplace = True)
df.drop_duplicates(inplace=True)
print("Length of dataframe after dropping duplicates: ", len(df))
df = df[df['OBS_RUN'] >= 0]


Length of dataframe for selected basin:  0
Length of dataframe after dropping duplicates:  0


/Users/omernivron/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
df.drop(columns= list_to_drop, inplace=True)

In [31]:
df.drop_duplicates(inplace=True)

In [32]:
df.drop(['MOD_RUN'], axis=1, inplace = True)

In [33]:
df1 = df[df['OBS_RUN'] >= 0]

In [39]:
df1['YR']

0          1980
1          1980
2          1980
3          1980
4          1980
           ... 
8236952    2014
8236953    2014
8236954    2014
8236955    2014
8236956    2014
Name: YR, Length: 8230212, dtype: int64

In [ ]:
context_channels = ['OBS_RUN',
                    'doy_cos','doy_sin',
                    'prcp(mm/day)', 
                    'srad(W/m2)',  
                    'tmax(C)',
                    'tmin(C)', 
                    'vp(Pa)'] 